Wahlpflichtfach Künstliche Intelligenz I: Praktikum

---

# 07 - Data Cleaning und Data Preparation

Data Cleaning und Data Preparation sind ein riesen Thema. Manche Leute behaupten, dass Data Scientists 80 % ihrer Zeit damit verbringen, ihre Daten zu bereinigen. Die Themen, die wir hier behandeln werden, sind 

* Umgang mit fehlenden Werten (Missing Values)
* Entfernen von Duplikaten
* Strukturierung von Daten
* Entfernen von Ausreißern (Outlier)
* Finden der richtigen Datentypen

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
%matplotlib inline

# Fehlende Werte (Missing Values)

Ein Sentinelwert wird verwendet, um fehlende Werte für Zahlen darzustellen. Eine spezielle Kombination von Bits steht für "Keine Zahl" (NaN). Dies kann man sich als das numerische Äquivalent von "Keine" vorstellen. In Python ist `NaN` durch die Pakete `NumPy` und `Pandas` verfügbar. Seit Pandas Version 1.0 werden fehlende Werte durch ein spezielles Objekt dargestellt: `pd.NA`.

Das mag auf den ersten Blick seltsam erscheinen, beginnt aber Sinn zu machen, wenn wir über die Semantik von `NaN` oder allgemeiner `NA` als Platzhalter für einen Wert, der __N__ot **A**vailable ist, nachdenken. Da `NA` einfach einen beliebigen Wert repräsentiert, den wir nicht kennen, wäre es falsch zu sagen, dass ein Wert, den wir nicht kennen, gleich einem anderen Wert ist, den wir nicht kennen. Daher kann `NA` nicht wirklich gleich irgendetwas sein.

Um explizit auf `NA` zu testen, benötigen wir eine eigene Funktion, die von `pandas` bereitgestellt wird.

In [ ]:
pd.isna(np.nan)

In [ ]:
pd.isna(pd.NA)

In [ ]:
pd.isna(42)

### Umgang mit fehlenden Werten

In [ ]:
ebola = pd.read_csv('data/07/ebola_country_timeseries.csv')
ebola.head()

In [ ]:
ebola['Cases_Guinea'].value_counts(dropna=False).head()

### Verwerfen

Der einfachste Weg, mit fehlenden Daten umzugehen, ist, sie einfach zu verwerfen. Dies kann jedoch zu einem immensen Datenverlust führen, je nachdem, wie die Daten organisiert sind.

In [ ]:
ebola.dropna()

In [ ]:
ebola.dropna(how='all')

### Auffüllen

Stattdessen können fehlende Werte aufgefüllt werden, damit der Rest der Daten brauchbar bleibt. Beachten Sie, dass dies immer Artefakte einführt.

Wir können mit einem konstanten Wert auffüllen.

In [ ]:
ebola.fillna(0).head()

Oder verwenden Sie einige fortgeschrittenere Strategien zur Berechnung der Daten, z. B. die Berechnung eines Mittelwerts pro Spalte. Dies kann durch jede einfache Summenstatistik ersetzt werden.

In [ ]:
ebola.mean(numeric_only=True)

In [ ]:
ebola.fillna(ebola.mean(numeric_only=True)).head()

Einige fortgeschrittenere Techniken, wie z. B. der Expectation Maximization (EM)-Algorithmus, existieren, sind aber nicht direkt in `pandas` implementiert. 

Beim Umgang mit fortlaufenden Daten kann es sinnvoll sein, fehlende Werte mit vorherigen oder nachfolgenden Werten aufzufüllen.

In [ ]:
ebola.fillna(method='ffill').head()

In [ ]:
ebola.fillna(method='bfill')

#### Fortgeschrittenes Auffüllen

Pandas bietet auch erweiterte Methoden zum Auffüllen fehlender Werte. Die Funktion [interpolate](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html#pandas-dataframe-interpolate) bietet verschiedene Möglichkeiten, die fehlenden Werte zu interpolieren.

In [ ]:
ebola['Cases_Guinea'].head()

In [ ]:
ebola['Cases_Guinea'].interpolate(method='quadratic').head()

### Berechnungen mit fehlenden Werten

Standardmäßig ist `NumPy` sehr streng bei Berechnungen mit `NA`-Werten. Jede Operation, die `NA` beinhaltet, wird `NA` ergeben. Das ist insofern korrekt, als dass der Endwert einer Operation wie `sum` nicht bekannt sein kann, wenn auch nur ein einziger Wert unbekannt ist.

In [ ]:
np.nansum([1, 2, np.nan, 3])

Aus praktischer Sicht ist dies jedoch nicht sehr sinnvoll. Daher verfolgt pandas den Ansatz, `NA`s gnädig zu ignorieren.

In [ ]:
ebola['Cases_Guinea'].sum()

Dieses Verhalten kann auf Wunsch geändert werden.

In [ ]:
ebola['Cases_Guinea'].sum(skipna=False)

## Entfernen von Duplikaten

Duplikate können als Teil ungeordneter Daten entstehen. Es ist wichtig, sie richtig zu identifizieren und zu beseitigen, damit sie unsere Statistiken nicht beeinflussen.

In [ ]:
df1 = pd.DataFrame({
    'a': [1, 1, 1, 2, 2, 2],
    'b': [10, 20, 30, 40, 50, 50],
})

df1

Prüfen, ob eine Zeile ein Duplikat ist.

In [ ]:
df1.duplicated()

Verwerfen Sie die doppelten Zeilen.

In [ ]:
df1.drop_duplicates()

Duplikatsuche auf eine Teilmenge der Spalten einschränken.

In [ ]:
df1.duplicated(subset='a')

In [ ]:
df1.drop_duplicates(subset='a')

## Data Preparation: Daten mit Pandas analysieren

## Datentypen

### Finden der richtigen Datentypen

Daten können in verschiedenen Maßstäben ausgedrückt werden. Sie müssen sicherstellen, dass Sie das Maßniveau finden, das sowohl semantisch als auch rechnerisch sinnvoll ist.

Ein kurzer Abstecher zu den Maßstäben
1. **Nominale Ebene** <br/>
   Zahlen stellen nur Kategorien dar und nichts weiter. <br/>
   Z.B.: Geschlechter, Farben<br/>
   Es können berechnet werden: absolute und relative Häufigkeiten, Modus   
   
1. **Ordinalebene** <br/>
   Die Reihenfolge hat eine Bedeutung.<br/>
   Z.B.: Schulnoten, Musik-Charts, Antworten auf einer Likert-Skala<br/>
   Sie können zusätzlich berechnen: kumulative Häufigkeiten, Median, Quantile   
   
1. **Intervallniveau** <br/>
   Gleiche Intervalle sollen die gleiche Bedeutung haben.<br/>
   Z.B.: Temperatur in Celsius, (Intelligenz-)Tests<br/>
   Sie können zusätzlich berechnen: Mittelwert, Standardabweichung   

1. **Verhältnisebene**<br/>
   Verhältnisse vermitteln Bedeutung und es gibt einen bestimmten 0-Punkt.<br/>
   Z.B.: Masse, Größe, Zeit, Geschwindigkeit<br/>
   Sie können berechnen: Variationskoeffizient $c = \frac{s}{\bar X}$, d.h. eine normierte Standardabweichung 


### Kategorische Daten
https://pandas.pydata.org/pandas-docs/stable/categorical.html

Die Verwendung eines kategorischen D-Typs hat mehrere Vorteile

* es hält den Speicherverbrauch niedrig
* es macht die Daten für numerische Modellierungsalgorithmen nutzbar
* Es signalisiert den Bibliotheken, die auf Pandas aufbauen, wie die Daten zu behandeln sind.
* es macht die Absicht klar, dass nur bestimmte Werte in einer Spalte erlaubt sind und wie sie sich zueinander verhalten

Die folgende `Serie` könnte perfekt mit Kategorien anstelle von Strings dargestellt werden.

In [ ]:
s = pd.Series(['a','b', 'b', 'a', 'c', 'c'])
s

In [ ]:
print(f'The string series is {s.nbytes} bytes big.')

Durch Angabe des `dtype` als "category" werden die Daten automatisch in eine kategoriale Skala umgewandelt.

In [ ]:
s = pd.Series(['a','b', 'b', 'a', 'c', 'c'], dtype='category')
s

In der Tat wird die `Serie` schon viel kleiner. Der Effekt wird bei größeren `Serien` stärker sein.

In [ ]:
print(f'The categorical series is {s.nbytes} bytes big.')

Kategoriale Daten werden unter der Haube mit numerischen Codes gespeichert, die den Kategorien zugeordnet sind.

In [ ]:
s.cat.categories

In [ ]:
s.cat.codes

Die Verwendung von `dtype='category'` erzeugt standardmäßig ungeordnete Kategorien.

In [ ]:
s.cat.ordered

Der Accessor `cat` erlaubt das Ändern, Umbenennen und Ordnen von Kategorien.

In [ ]:
s.cat.categories

In [ ]:
s.cat.rename_categories(['x', 'y', 'z'])

Eine kategoriale Reihe kann auch aus `pd.Categorical` erstellt werden. Damit können Sie die Kategorien und die Reihenfolge explizit festlegen.

In [ ]:
pd.Categorical(['a', 'b', 'c', 'a'], categories=['b', 'c'],ordered=False)

Das `Categorical`-Objekt kann dann an den `Series`-Konstruktor übergeben werden, um eine echte `Series` zu erhalten.

In [ ]:
cat_series = pd.Series(
    pd.Categorical(['a', 'b', 'c', 'a'], categories=['b', 'c', 'a'],
                         ordered=False)
)
cat_series

### Geordnete Kategorien

Was bedeutet es, geordnete Kategorien zu haben?

In [ ]:
cat_series2 = pd.Series(
    pd.Categorical(['c', 'a', 'c', 'b'], categories=['b', 'c', 'a'],
                         ordered=False)
)
cat_series2

In [ ]:
cat_series == cat_series2

In [ ]:
cat_series > cat_series2

In [ ]:
cat_series

In [ ]:
cat_series.mode()

In [ ]:
cat_series.max()

Diese Semantik geht verloren, wenn Sie die atomaren Werte herausziehen. Nur die `Serie` ist kategorisch, nicht die einzelnen Einträge.

In [ ]:
cat_series.iloc[0], type(cat_series.iloc[0])

In [ ]:
cat_series.iloc[0] < cat_series.iloc[1]

Nun das Gleiche für eine **geordnete** kategoriale `Serie`.

In [ ]:
cat_ordered_series = pd.Series(
    pd.Categorical(['a', 'b', 'c', 'a'], categories=['b', 'c', 'a', 'd'],
                         ordered=True)
)
cat_ordered_series

In [ ]:
cat_ordered_series2 = pd.Series(
    pd.Categorical(['c', 'a', 'c', 'b'], categories=['b', 'c', 'a', 'd'],
                    ordered=True)
)
cat_ordered_series2

In [ ]:
cat_ordered_series > cat_ordered_series2

In [ ]:
cat_ordered_series.max()

In [ ]:
cat_ordered_series == cat_ordered_series2

Der Median funktioniert nicht bei den kategorialen Reihen, kann aber mit den Codes berechnet werden.

In [ ]:
cat_ordered_series

In [ ]:
cat_ordered_series.median()

In [ ]:
cat_ordered_series.cat.codes.median()

Wenn Sie vorhandene Daten in einen kategorischen Typ umwandeln und die Kategorien und die Reihenfolge angeben möchten, können Sie mit `pd.CategoricalDtype` einen eigenen kategorischen Datentyp erstellen. Es funktioniert auf die gleiche Weise wie `pd.Categorical`, nur dass Sie die Daten nicht übergeben. Der neu erstellte Datentyp kann dann in einem `astype()`-Cast verwendet werden.

In [ ]:
series = pd.Series(['a', 'b', 'c', 'a'])
series

In [ ]:
from pandas.api.types import CategoricalDtype

cat_type = CategoricalDtype(categories=['b', 'c', 'a'],
                             ordered=True)
cat_type

In [ ]:
series.astype(cat_type)

Schauen wir uns nun einen Datensatz aus der realen Welt und einige Diskretisierungstechniken an. Der Titanic-Datensatz enthält Merkmale über Passagiere der tragischen Titanic-Reise. Eine übliche einführende Übung zum maschinellen Lernen ist die Vorhersage des Überlebens der Passagiere auf der Grundlage der Merkmale (siehe https://www.kaggle.com/c/titanic/data).

In [ ]:
titanic = pd.read_csv('data/07/titanic.csv')
titanic.head()

In [ ]:
titanic.dtypes

Wir nehmen alle Spalten in die Beschreibung auf, da "Objekt"-Spalten anders beschrieben werden als "numerische" Spalten und standardmäßig von der Beschreibung ausgeschlossen sind.

In [ ]:
titanic.describe(include='all')

Erweitern wir den Einschiffungshafen um den vollständigen Namen, um die Dinge etwas lesbarer zu machen. Dazu verwenden wir eine einfache Zusammenführungsoperation (mehr dazu später).

In [ ]:
embarked_map = pd.DataFrame({'Embarked': ['C', 'Q', 'S'],
                             'EmbarkedLong': ['Cherbourg', 'Queenstown', 'Southampton']})
embarked_map

In [ ]:
titanic = titanic.merge(embarked_map).sort_values(by='PassengerId')
titanic.head()

In [ ]:
titanic.dtypes

Da die Spalte "EmbarkedLong" nur drei unterschiedliche Werte hat, ist es sinnvoll, sie mit Kategorien darzustellen.

In [ ]:
titanic['EmbarkedLong'].unique()

In [ ]:
titanic['EmbarkedLong'] = titanic['EmbarkedLong'].astype('category')
titanic['EmbarkedLong'].head()

In [ ]:
titanic.dtypes

Die Beschreibung für eine kategorische Spalte ist die gleiche wie für eine `Objekt`-Spalte.

In [ ]:
titanic['EmbarkedLong'].describe()

## Diskretisieren kontinuierlicher Werte (Tiling)
Manchmal ist es sinnvoll, numerische in kategorische Daten umzuwandeln. Zum Beispiel spielt bei manchen Problemen das genaue Alter einer Person keine Rolle, sondern nur, ob die Person minderjährig ist oder nicht. Dieser Konvertierungsprozess wird Kacheln genannt.

https://pandas.pydata.org/pandas-docs/stable/basics.html#discretization-and-quantiling

In [ ]:
titanic['Age'].describe()

Mit `cut` können wir numerische Werte diskretisieren.

In [ ]:
titanic['Age'].head(7)

In [ ]:
pd.cut(titanic['Age'], bins=3).head(7)

Standardmäßig wird `cut()` die Daten in gleich große Intervalle aufteilen. Da dies nur selten sinnvoll ist, können wir die Bin-Kanten selbst festlegen.

In [ ]:
pd.cut(titanic['Age'], bins=[0, 17, 67, 80], include_lowest=True).head(7)

In [ ]:
pd.cut(titanic['Age'], bins=[0, 17, 67, 80]).value_counts()

Wenn Sie die Bereichsgrenzen manuell einstellen, achten Sie darauf, dass Sie den gesamten Bereich abdecken, da Werte, die nicht in ein Intervall fallen, auf NA gesetzt werden.

In [ ]:
pd.cut(titanic['Age'], 
       bins=[64, 66, 67, 80],
       labels=['child', 'grown-up', 'senior']).head(7)

In [ ]:
titanic['Age_coarse'] = pd.cut(titanic['Age'], bins=[0, 17, 67, 80], labels=['child', 'grown-up', 'senior'])
titanic['Age_coarse']

Eine verwandte Funktion ist `qcut()`, die an Quantilen schneidet.

In [ ]:
pd.qcut(titanic['Age'], 4).head()

### Konvertierung in numerische Daten

Manchmal werden numerische Daten irgendwie verdreht. pd.to_numeric" behandelt diese Fälle und wandelt alles automatisch in den entsprechenden Typ um.

In [ ]:
numeric_data = pd.read_csv('data/07/numeric_data.csv')
numeric_data

In [ ]:
numeric_data.dtypes

In [ ]:
numeric_data['C'].sum()

In [ ]:
numeric_data['B'].astype('int')

In [ ]:
numeric_data['A'].astype('float')

In [ ]:
pd.to_numeric(numeric_data['A'], errors='ignore')

In [ ]:
pd.to_numeric(numeric_data['A'], errors='coerce')

In [ ]:
pd.to_numeric(numeric_data['B'], errors='coerce')

In [ ]:
pd.to_numeric(numeric_data['C'], errors='coerce')

`to_numeric()` funktioniert nur bei Serien, aber zum Glück können wir `apply()` verwenden!

In [ ]:
numeric_data

In [ ]:
numeric_data.apply(pd.to_numeric, errors='coerce').dtypes #keyword-arguments are passed to the respective function

In [ ]:
isinstance(np.nan, float)

## Plotten mit Pandas

Pandas bietet einige Plotting-Funktionen an, die auf den entsprechenden Funktionen von matplotlib aufbauen und diese Funktionen intern selbst aufrufen. Um deren Verhalten zu ändern, kann man ihnen eine Achse an ein matplotlib-Objekt übergeben: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html

In [ ]:
titanic['Age'].hist(density=True)

In [ ]:
plt.hist(titanic['Age'].dropna().values, density=True)
plt.grid()

In [ ]:
fig, ax = plt.subplots()
titanic['Age'].hist(density=True, ax=ax)
ax.set_xlabel('Age')

In [ ]:
titanic['Age_coarse'].value_counts()

In [ ]:
titanic['Age_coarse'].value_counts().plot(kind='pie')

In [ ]:
coarse_age_series = titanic['Age_coarse'].cat.add_categories(['unknown'])
coarse_age_series

In [ ]:
coarse_age_series.cat.categories

In [ ]:
coarse_age_series = coarse_age_series.fillna('unknown')
coarse_age_series

In [ ]:
coarse_age_series.value_counts()

In [ ]:
coarse_age_series.value_counts().plot(kind='pie')

In [ ]:
coarse_age_series.value_counts().plot(kind='barh')

Die Funktion `plot()` funktioniert auch mit DataFrames.

In [ ]:
titanic[['Age', 'Fare']].plot(kind='scatter', x='Age', y='Fare')

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html

## Mergen von DataFrames

Das Merging ist ein Konzept, das häufig in relationalen Datenbanken verwendet wird. Es erlaubt, mehrere Tabellen zu einer zusammenzufassen, indem die Spalten in Bezug auf die Werte in einer speziellen Schlüsselspalte verbunden werden. Es gibt verschiedene Möglichkeiten, wie dies erreicht werden kann.

Die Funktion `DataFrame.merge` bietet diese aus SQL (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas-dataframe-merge) entlehnten Funktionalitäten.

In [ ]:
df1 = pd.DataFrame({'A': [1, 2, 3, 4],
                    'B': [0, np.pi, 2 * np.pi, 3 * np.pi],
                    'C': ['mouse', 'cat', 'dog', 'fish']})
df1

In [ ]:
df2 = pd.DataFrame({'C': ['mouse', 'horse', 'lizard', 'fish'],
                    'D': [1.0, 1.7, 3.0, 2.1],
                    'E': [1, np.e, np.e ** 2, np.e ** 3]})
df2

### Inner Join

Der innere Join nimmt die Schnittmenge der Schlüssel.

In [ ]:
df1.merge(df2, how='inner')

### Left Outer Join

Der Left Outer Join behält alle Werte aus der linken Tabelle (derjenigen, auf der merge aufgerufen wird) und verwendet `NaN`, wenn in der rechten Tabelle die entsprechenden Zeilen fehlen.

In [ ]:
df1.merge(df2, how='left')

### Right Outer Join
Der Right Outer Join funktioniert genauso wie der Left Outer Join, aber statt aller Schlüssel aus der linken Tabelle werden alle Schlüssel aus der rechten Tabelle verwendet.

In [ ]:
df1.merge(df2, how='right')

### Outer Join
Die Outer Join verwendet alle Schlüssel, die sowohl in der linken als auch in der rechten Tabelle vorhanden sind. Fehlende Zeilen in einer der Tabellen werden mit `NaN` aufgefüllt.

In [ ]:
df1.merge(df2, how='outer')

### Überlappende Spaltennamen

In [ ]:
df2 = df2.rename(columns={'D': 'A'})
df2

In [ ]:
df1

Wenn es nicht implizit klar ist, auf welcher Spalte der Join stattfinden soll, müssen wir Pandas sagen, welche Spalte es verwenden soll. Es kann den Join auch auf mehreren Spalten durchführen, aber dafür müssen die dtypes der übereinstimmenden Spalten innerhalb der beiden DataFrames gleich sein.

In [ ]:
df1.merge(df2, how='inner')

Wir können Pandas mit dem Schlüsselwort-Argument `on` explizit mitteilen, auf welcher Spalte die Verknüpfung stattfinden soll. Mit dem Parameter `suffixes` können wir steuern, wie überlappende Spaltennamen im verbundenen DataFrame geändert werden sollen.

In [ ]:
df1.merge(df2, how='inner', on='C', suffixes=('_from_df1', '_from_df2'))

In [ ]:
df1['A'] = df1['A'].astype(float)
df1.merge(df2, how='inner', on=['A', 'C'])

## Arbeiten mit Zeitreihendaten

https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html  
Der grundlegendste Baustein von Zeitreihendaten in Pandas ist der `Timestamp`. Er repräsentiert einen Moment in der Zeit mit der Genauigkeit einer Nanosekunde. Er wird ergänzt durch `Timedelta`, das eine Zeitspanne wie "ein Monat" repräsentiert, ohne an ein Datum gebunden zu sein, und `Period`, das eine Kombination aus beiden ist, wie "Juni 2018". Dabei muss `Period` eine gewisse Regelmäßigkeit haben, wie z. B. jeden Monat.

### Timestamps (Zeitstempel)
Timestamps können einfach aus menschenlesbaren Strings mit `pd.datetime` erstellt werden.

In [ ]:
pd.to_datetime('2020-06-09')

In [ ]:
pd.to_datetime('9th June 20')

In [ ]:
pd.to_datetime('06.09.2020')

Für Nicht-Amerikaner und Leute, die denken, dass der Tag vor dem Monat kommen sollte.

In [ ]:
pd.to_datetime('09.06.2020', dayfirst=True)

In [ ]:
pd.to_datetime('2020-06-09 14:45')

In [ ]:
date = pd.to_datetime('2020-06-09 14:45:30.600700800')
date

`Timestamps` stellen alle Informationen über Attribute zur Verfügung.

In [ ]:
date.year

In [ ]:
date.month

In [ ]:
date.day

In [ ]:
date.second

In [ ]:
date.microsecond

In [ ]:
date.nanosecond

Timestamps können verglichen werden:

In [ ]:
date1 = pd.to_datetime('2020-06-09 14:45')
date2 = pd.to_datetime('2020-06-09 14:46')
date1 < date2

Wenn eine Serie übergeben wird, gibt `to_datetime()` eine Serie (mit demselben Index) zurück, während eine Liste in einen DatetimeIndex umgewandelt wird:

In [ ]:
pd.to_datetime(pd.Series(['Jul 31, 2009', '2010-01-10', None]), format='mixed')

In [ ]:
pd.to_datetime(['2005/11/23', '2010.12.31'], format='mixed')

Timestamps" können mit einem speziellen Satz von Symbolen formatiert werden. Alle diese Symbole finden Sie hier https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
date.strftime('Today is a %A in %B')

In [ ]:
pd.to_datetime('12-11-2010 00:00', format='%d-%m-%Y %H:%M')

### DatetimeIndex

Timestamps können zur Indizierung von Daten verwendet werden.

In [ ]:
index = pd.DatetimeIndex(['2020-06-16', '2020-06-23',
                          '2020-06-30', '2020-07-07',
                          '2020-07-14'])
schedule = pd.Series(['Statistical Visualization', 'SciPy and Statistical Modeling I',
                      'Statistical Modeling II', 'Creating Experiments',
                      'Performance Optimization'], index=index)
schedule

In [ ]:
schedule['2020-06-10':'2020-06-30']

So wie es NaN für Zahlen gibt, gibt es NaT (Not-A-Time) für Timestamps:

In [ ]:
dt = pd.to_datetime(['2009/07/31', 'asd'], errors='coerce')
dt

`isnull()` prüft auf fehlende Daten in DatetimeIndex-Objekten (NaN in numerischen Arrays, None oder NaN in Objekt-Arrays, NaT in datetimelike):

In [ ]:
dt.isnull()

### Lesen von Zeitreihendaten

Lesen Sie die Daten, die im Format "Zeilen mit fester Breite" formatiert sind.

In [ ]:
ts = pd.read_fwf('data/07/ao_monthly.txt', header=None, index_col=0)
ts.head()

Dies erzeugt einen Integer-Index anstelle des gewünschten `DateTimeIndex`.

In [ ]:
ts.index

In [ ]:
ts = pd.read_fwf('data/07/ao_monthly.txt', header=None, index_col=0,
                 parse_dates=[[0, 1]], infer_datetime_format=True)
ts.head()

In [ ]:
ts.index

In [ ]:
ts.plot()

Jetzt, da unsere Reihe durch Zeitstempel indiziert ist, können wir mit zeitbezogener Semantik aggregieren.

In [ ]:
ts.index.year

In [ ]:
ts.groupby(ts.index.year).mean().head()

In [ ]:
ts.groupby(ts.index.year).mean().plot(marker='o');

Mit `pd.Grouper()` können wir komplexere Gruppierungen festlegen.

In [ ]:
ts.groupby(pd.Grouper(freq='5Y')).mean().head()

In [ ]:
ts.groupby(pd.Grouper(freq='d')).mean().head()

### Resampling (Neuabtastung)

Wenn Sie mit der Häufigkeit, mit der Ihre Daten abgetastet werden, nicht zufrieden sind, können Sie die Abtastfrequenz ändern.

In [ ]:
nineteenfifty = ts[ts.index.year == 1950]
nineteenfifty.head()

In [ ]:
nineteenfifty.plot(marker='o');

In [ ]:
nineteenfifty.asfreq('12D', method='ffill').head()

In [ ]:
nineteenfifty.asfreq('12D', method='ffill').plot(style='--o');

In [ ]:
fig, ax = plt.subplots(nrows=2, sharex=True)

# row 1
nineteenfifty.asfreq('12D').plot(ax=ax[0], style='-o') # no fill
# row 2
nineteenfifty.asfreq('12D', method='ffill').plot(ax=ax[1], marker='o') # forward-fill
nineteenfifty.asfreq('12D', method='bfill').plot(ax=ax[1], style='--o') # back-fill
nineteenfifty.plot(ax=ax[1], style='o') # original

ax[0].legend(['no fill'])
ax[1].legend(['forward-fill', 'back-fill', 'original']);

Downsampling kann durch Angabe einer kleineren Frequenz erfolgen.

In [ ]:
nineteenfifty.asfreq('3M', method='ffill').plot(marker='o');

In [ ]:
fig, ax = plt.subplots()

nineteenfifty.asfreq('3M', method='ffill').plot(marker='o', ax=ax) # downsampled
nineteenfifty.plot(ax=ax, style='--o') # original

ax.legend(['3 Month', 'original']);

Resampling kann auch mit Aggregation durch `resample()` kombiniert werden. Schauen wir uns einige Aktiendaten an, um dies zu veranschaulichen.

In [ ]:
yahoo = pd.read_csv('data/07/yahoo_stock.csv', index_col=0, parse_dates=True)
yahoo.head()

In [ ]:
ts = yahoo['Close']
ts.plot();

http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

In [ ]:
ts.plot(alpha=0.5, style='-')
ts.resample('BA').mean().plot(style=':')
ts.asfreq('BA').plot(style='--');
plt.legend(['input', 'resample', 'asfreq'], loc='upper left');

### Verschieben und Differenzieren

In [ ]:
ts_resampled = ts.asfreq('D', method='ffill')

Das Verschieben von Daten in der Zeit kann auf zwei Arten erfolgen. Mit "Shift" werden die Daten tatsächlich verschoben. Dabei entstehen auf der einen Seite fehlende Werte und auf der anderen Seite gehen Daten verloren. Im Gegensatz dazu verschiebt `tshift` nur den Zeitindex der Daten und nicht die Daten selbst.

In [ ]:
fig, axes = plt.subplots(nrows=3, sharey=True, figsize=(10, 8))

ts_resampled.plot(ax=axes[0], title='Original')
ts_resampled.shift(365).plot(ax=axes[1], title='shift(365)')
ts_resampled.shift(365, "d").plot(ax=axes[2], title='tshift(365)')

axes[0].axvline('2011', alpha=0.5, color='r', linewidth=3)
axes[1].axvline('2011', alpha=0.5, color='r', linewidth=3)
axes[2].axvline('2011', alpha=0.5, color='r', linewidth=3)

plt.tight_layout()

Die Verschiebung ist nützlich für Berechnungen, die Werte über Zeitschritte hinweg vergleichen. Ein Beispiel ist das Differenzieren, um den Trend in der Zeitreihe zu entfernen.

In [ ]:
(ts_resampled - ts_resampled.shift(periods=1)).plot()

Für die Differenzierung stellt Pandas die komfortable Methode `diff` zur Verfügung.

In [ ]:
ts_resampled.diff(periods=1).plot()

### Window-Funktionen

Window-Funktionen sind ähnlich wie `groupby`, da sie die Daten in verschiedene Gruppen basierend auf einem sich ändernden Fenster aufteilen. Die Punkte in jedem Fenster werden mithilfe einer zusammenfassenden Statistik aggregiert und dann wieder zu einer Zeitreihe kombiniert.

#### Rollendes Window

Ein rollendes Window ist das Standardbeispiel für eine Window-Funktion. Es verschiebt ein Fenster mit fester Größe über die Zeitreihe.

In [ ]:
ts_resampled.plot()
ts_resampled.rolling(365).mean().plot()

Wenn Sie `center=True` einstellen, wird der Punkt, der aggregiert und in die neue Serie eingefügt wird, aus der Mitte des Fensters und nicht von seinem Ende aus stammen. 

In [ ]:
ts_resampled.plot()
ts_resampled.rolling(365, center=True).mean().plot()

### Expandierendes Window

Ein expandierendes Window hat nur eine minimale Größe. Dann wird es mit jedem Schritt größer, wobei alle vorherigen Werte berücksichtigt werden. Dies ist sinnvoll, wenn Ihre Zeitreihe einen stationären Wert misst, der nur um einen Mittelwert schwankt.

In [ ]:
ts_resampled.plot()
ts_resampled.expanding(min_periods=365).mean().plot()

### Exponential gewichtete Window

Ein exponentiell gewichtetes Window funktioniert wie ein expandierendes Fenster, gibt aber neueren Datenpunkten eine exponentiell höhere Gewichtung in allen Berechnungen. Es kann also als eine glatte Version eines rollenden Window betrachtet werden.

In [ ]:
ts_resampled.plot()
ts_resampled.ewm(com=50.5, min_periods=5).mean().plot()

### Timedeltas und Perioden

Timedeltas können zu Timestamps hinzugefügt werden.

In [ ]:
delta = pd.to_timedelta('1 day')
delta

In [ ]:
schedule

In [ ]:
schedule.index += delta
schedule

In [ ]:
pd.to_datetime('2019-08-15') - pd.to_datetime('2018-06-04')

In [ ]:
schedule.index += (pd.to_datetime('2019-08-15') - pd.to_datetime('2018-06-05'))
schedule.index = schedule.index.date
schedule

Die Kombination von Timestamps und Timedeltas ermöglicht eine schöne Arithmetik mit Datumsangaben:

In [ ]:
friday = pd.Timestamp('2018-01-05')
saturday = friday + pd.to_timedelta('1 day')
saturday, saturday > friday, saturday - friday

Es gibt sogar Geschäftstage in Pandas (Freitag --> Montag):

In [ ]:
friday = pd.Timestamp('2018-01-05')
monday = friday + pd.offsets.BDay()
monday

### date_range

Ein bequemerer Weg, einen solchen Index zu erstellen, ist die Verwendung von `date_range`.  
Perioden" gibt an, wie viele Einträge wir wollen, alternativ könnten wir einen expliziten "Stopp" setzen. `freq` gibt an, wie die Einträge beabstandet sind. Die vollständige Liste der möglichen Offsets finden Sie hier http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases. Die Syntax ist also sehr ähnlich zu "range(start, stop, step)".

In [ ]:
index = pd.DatetimeIndex(['2020-06-16', '2020-06-23',
                          '2020-06-30', '2020-07-07',
                          '2020-07-14'])
schedule = pd.Series(['Statistical Visualization', 'SciPy and Statistical Modeling I',
                      'Statistical Modeling II', 'Creating Experiments',
                      'Performance Optimization'], index=index)
schedule

In [ ]:
index = pd.date_range('2018-06-04', periods=5, freq='W')
index

Beachten Sie, dass `freq='W'` nicht eine einfache wöchentliche Häufigkeit bedeutet, sondern vielmehr **das Ende der Woche für alle diese Daten**.

In [ ]:
index = pd.date_range('2018-06-04', periods=5, freq='7D')
index

Pandas ist schlau im Ableiten von Frequenzen:

In [ ]:
tmp = pd.DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-05'], freq='infer')
tmp

In [ ]:
ts = pd.Series(range(len(tmp)), index=tmp)
ts

In [ ]:
ts.resample('D').sum().index

Alternativ könnten wir auch einen `period`-Index verwenden, um zu signalisieren, dass ein Topic zu einer ganzen Woche gehört.

In [ ]:
prd = pd.Period('2018-06-04', '7D')
prd

In [ ]:
prd.freq

In [ ]:
index = pd.period_range('2018-06-04', periods=5, freq='W')
schedule = pd.Series(['Statistical Visualization', 'SciPy and Statistical Modeling I',
                      'Statistical Modeling II', 'Creating Experiments',
                      'Performance Optimization'], index=index)
schedule

Sie können einfach zwischen `Timestamp` und Periode konvertieren.

In [ ]:
schedule = schedule.to_timestamp()
schedule

In [ ]:
schedule.to_period(freq='W')

In [ ]:
prd

In [ ]:
prd.to_timestamp().to_period(freq='2D')

### Zugriff auf Werte in Serien

Für Serien und Indizes, die von normalen NumPy-Arrays unterstützt werden, gibt Series.array ein neues arrays.PandasArray zurück, das eine dünne (nicht kopierbare) Hülle um ein numpy.ndarray ist. PandasArray ist für sich genommen nicht besonders nützlich, aber es bietet die gleiche Schnittstelle wie jedes Erweiterungsarray, das in Pandas oder von einer Bibliothek eines Drittanbieters definiert wurde.

In [ ]:
idx = pd.period_range('2000', periods=4)
idx

In [ ]:
idx.array

In [ ]:
pd.Series([1, 2, 3]).array

In [ ]:
idx.to_numpy()

In [ ]:
type(idx.to_numpy()[0])

Erste Info zu allem, was mit Zeitreihen zu tun hat: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

Zusätzlich: Ein komplettes Tutorial zur Zeitreihenanalyse. Es beinhaltet den Umgang mit Zeitzonen sowie grundlegende Zeitreihenvorhersage und -klassifikation.

## Explorative Datenanalyse

Die explorative Datenanalyse (EDA) beschreibt den Prozess des Aufbaus einer Intuition für unsere Daten. Er wird durch eine Kombination von Datentransformationen und Visualisierungen erreicht. Typische Schritte im Prozess der EDA sind:


1. Recherchieren der Felder des Datensatzes 
2. Hypothesen bilden/Untersuchungsthemen entwickeln, die untersucht werden sollen 
3. Daten zusammenstellen 
3. Qualität der Daten beurteilen 
4. Daten profilieren 
5. Untersuchen Sie jede einzelne Variable im Datensatz 
6. Beurteilen Sie die Beziehung zwischen jeder Variable und dem Ziel 
7. Beurteilen Sie Wechselwirkungen zwischen den Variablen 
8. Daten über viele Dimensionen hinweg erforschen 

EDA ist sehr wichtig, da wir nicht beurteilen können, ob unsere Modellierung Sinn macht, wenn wir kein Gespür für unsere Daten haben. Während jede Analyse mit EDA beginnt, werden Sie immer wieder zu ihr zurückkehren, wenn Sie neue Ergebnisse aus der Modellierung erhalten.

Hier stellen wir Pivot-Tabellen als eine einfache Möglichkeit vor, die Beziehungen zwischen Variablen zu untersuchen.

## Pivot für die Analyse 

Letztes Mal haben wir Pivot-Tabellen als eine Möglichkeit vorgestellt, unordentliche Daten neu zu strukturieren. Ursprünglich sind sie jedoch eine Operation, um tabellarische Zusammenfassungen von Daten zu erstellen. Sie können als bequeme Abkürzung für ein zweidimensionales Groupby verwendet werden. Schauen wir uns zuerst ein normales Groupby an:

In [ ]:
titanic.groupby('Sex').mean(numeric_only=True)

Nehmen wir an, wir wollen den Einfluss von Geschlecht und Passagierklasse auf die Überlebensrate im Titanic-Datensatz analysieren.

In [ ]:
titanic.groupby(['Sex', 'Pclass'])['Survived'].mean()

Wenn Sie den Index zurücksetzen, sieht das Ganze etwas schöner aus.

In [ ]:
titanic.groupby(['Sex', 'Pclass'])['Survived'].mean().reset_index()

Für Leute, die an das tidy Format gewöhnt sind, ist dies intuitiv zu lesen. Vielleicht möchten Sie aber trotzdem die zweite Variable in den Spaltenüberschriften haben. Dies nennt man eine "Pivot-Tabelle".

In [ ]:
titanic.groupby(['Sex', 'Pclass'])['Survived'].mean().unstack()

Um genau das zu tun, bietet pandas eine Abkürzung an.

In [ ]:
titanic.pivot_table(values='Survived', index='Sex', columns='Pclass')

Pivot-Tabellen können auch die Ränder, d. h. die über Zeilen und Spalten aggregierten Werte, enthalten.

In [ ]:
titanic.pivot_table(values='Survived', index='Sex', columns='Pclass', margins=True)

Standardmäßig aggregiert `pivot_table` mit dem Mittelwert, aber wir können auch alle in `groupby` verfügbaren Funktionen auswählen oder unsere eigenen verwenden.

In [ ]:
titanic.pivot_table(values='Fare', index='Sex', columns='Pclass', aggfunc=[min, max])

Die Kombination von mehr als zwei Variablen ist ebenfalls möglich, indem sie entweder in den Zeilen oder in den Spalten gestapelt werden.

In [ ]:
titanic.pivot_table(values='Fare', index=['Sex', 'EmbarkedLong'], columns='Pclass',aggfunc='mean')

In [ ]:
titanic['Age_coarse'] = pd.cut(titanic['Age'], bins=[0, 17, 67, 80], labels=['child', 'grown-up', 'senior'])
titanic['Age_coarse']

Das Tool [`pivottablejs`](https://github.com/nicolaskruchten/pivottable) ermöglicht es Ihnen, Daten mit Pivotables per Drag'n'Drop schnell zu erkunden. Bei der Verwendung eines solchen grafischen Werkzeugs sollten Sie darauf achten, dass Sie die interessanten Dinge in Code umwandeln, damit sie nach dem Schließen des Notizbuchs nicht verloren gehen.

In [ ]:
from pivottablejs import pivot_ui
pivot_ui(titanic)

### Profiling

Wenn man eine explorative Datenanalyse durchführt, müssen viele Aufgaben jedes Mal neu durchgeführt werden, damit sie automatisiert werden können. Werkzeuge wie `pandas_profiling` können Summeries erstellen, die Einblicke in viele Standardfragen geben, die Sie an einen Datensatz stellen können. Allerdings kommt mit der Abstraktion auch weniger Flexibilität, so dass Werkzeuge wie dieses immer nur einen Teil Ihrer Arbeit erledigen und manchmal vielleicht gar nicht das tun, was Sie wollen.

In [ ]:
from ydata_profiling import ProfileReport
ProfileReport(titanic)

Im folgenden Tutorial erfahren Sie mehr über Werkzeuge und Prozesse der explorativen Datenanalyse.

---

Wahlpflichtach Künstliche Intelligenz I: Praktikum